In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import xgboost as xgb
from xgboost import XGBClassifier
import time

In [2]:
df_tr = pd.read_csv('train.csv')
df_ts = pd.read_csv('test.csv')
df_s = pd.read_csv('sample_submission.csv')

In [3]:
# Check the target column type
print(df_tr['Target'].dtype)

# If the target is categorical, convert it to numeric using LabelEncoder
if df_tr['Target'].dtype == 'object':
    le = LabelEncoder()
    df_tr['Target'] = le.fit_transform(df_tr['Target'])

# Split data into features (X) and target (y)
X = df_tr.drop(columns=['Target','id'], axis=1)
y = df_tr['Target']

# Apply Quantile Transformer to X
#scaler = QuantileTransformer()
#X_scaled = scaler.fit_transform(X)
X_scaled =X


# Define the parameter grid for RandomizedSearchCV
param_grid = {
    'n_estimators': np.arange(50, 500, 50),
    'learning_rate': np.linspace(0.01, 0.3, 30),
    'max_depth': np.arange(3, 10, 1),
    'subsample': np.linspace(0.5, 1.0, 6),
    'colsample_bytree': np.linspace(0.5, 1.0, 6)
}

object


In [4]:
# Set up the XGBClassifier
xgb_model = XGBClassifier()

# Set up KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

setup=time.time()
# Set up the RandomizedSearchCV with k-fold cross-validation
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    n_iter=50,
    scoring='accuracy',
    cv=kf,
    verbose=1,
    random_state=42,
    n_jobs=-1
)
setup_end=time.time()
print(setup_end-setup)


0.0


In [5]:
# Fit the RandomizedSearchCV with the training data
fitting_start=time.time()
random_search.fit(X_scaled, y)
fitting_end=time.time()
print(fitting_end-fitting_start)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
842.12930560112


In [6]:
# Get the best parameters
bparam_start=time.time()
best_params = random_search.best_params_
print("Best parameters found: ", best_params)
bparam_end=time.time()
print(bparam_end-bparam_start)

Best parameters found:  {'subsample': 0.5, 'n_estimators': 400, 'max_depth': 5, 'learning_rate': 0.08999999999999998, 'colsample_bytree': 0.8}
0.0


In [7]:
# Train the final XGBClassifier with the best parameters
train_start=time.time()
best_xgb_model = XGBClassifier(**best_params)
best_xgb_model.fit(X_scaled, y)
train_end=time.time()
print(train_end-train_start)

11.206517457962036


In [8]:
# Split the scaled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Predict on the test set using XGBoost
y_pred_xgb = best_xgb_model.predict(X_test)

In [9]:
# Evaluate the XGBoost model
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print("XGBoost accuracy on test set: ", accuracy_xgb)

XGBoost accuracy on test set:  0.8633037114479875


In [10]:
df_ts_drop=df_ts.drop(columns=['id'], axis=1)

In [11]:
# Apply Quantile Transformer to X
scaler = QuantileTransformer()
df_ts_scaled = scaler.fit_transform(df_ts_drop)

In [12]:
L_Pred = best_xgb_model.predict(df_ts_drop)
# Making Submission File 
df_s['Target'] = le.inverse_transform(L_Pred)
df_s.to_csv('Submission_refined.csv', index=False)

In [13]:
df=pd.read_csv('Submission_refined.csv')

In [14]:
df.head()

,id,Target
0,76518,Dropout
1,76519,Graduate
2,76520,Graduate
3,76521,Graduate
4,76522,Enrolled
